In [963]:
# from sklearn import svm,metrics
# from sklearn.neighbors import KNeighborsClassifier # 븐류
# from sklearn.neighbors import KNeighborsRegressor # 회귀- 예측
from sklearn.linear_model import LinearRegression # 선형회귀
from sklearn.preprocessing import PolynomialFeatures # 다항특성을 만들어주는 라이브러리
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import matplotlib
import glob, os.path, re

matplotlib.rcParams['axes.unicode_minus'] = False  # 마이너스 표시 해결
# 한글설정
matplotlib.rcParams['font.family'] = 'Malgun Gothic' # windows 사용자
# matplotlib.rcParams['font.family'] = 'AppleGothic Gothic'

In [964]:
df_test = pd.read_csv('titanic_test.csv')
df_test

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,body,home.dest
0,3,0,"Rekic, Mr. Tido",male,38.0,0,0,349249,7.8958,NaN,S,NaN,NaN
1,2,1,"Phillips, Miss. Alice Frances Louisa",female,21.0,0,1,S.O./P.P. 2,21.0000,NaN,S,NaN,"Ilfracombe, Devon"
2,2,0,"Jacobsohn, Mr. Sidney Samuel",male,42.0,1,0,243847,27.0000,NaN,S,NaN,London
3,3,0,"Risien, Mr. Samuel Beard",male,NaN,0,0,364498,14.5000,NaN,S,NaN,NaN
4,2,0,"Denbury, Mr. Herbert",male,25.0,0,0,C.A. 31029,31.5000,NaN,S,NaN,"Guernsey / Elizabeth, NJ"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
388,3,0,"Karlsson, Mr. Julius Konrad Eugen",male,33.0,0,0,347465,7.8542,NaN,S,NaN,NaN
389,2,1,"Ware, Mrs. John James (Florence Louise Long)",female,31.0,0,0,CA 31352,21.0000,NaN,S,NaN,"Bristol, England / New Britain, CT"
390,3,1,"O'Keefe, Mr. Patrick",male,NaN,0,0,368402,7.7500,NaN,Q,NaN,NaN
391,3,0,"Tobin, Mr. Roger",male,NaN,0,0,383121,7.7500,F38,Q,NaN,NaN


In [965]:
# 타이나닉의 생존자 분류 모델을 구현하시오.
# target : survived
# data : 그 외

# 데이터 확인: 0,nan,타입 확인

# 분류 : knn, 로지스틱, 랜덤포레스트

df_train = pd.read_csv('titanic_train.csv')
df_train.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,body,home.dest
0,2,1,"Mellinger, Miss. Madeleine Violet",female,13.0,0,1,250644,19.5000,NaN,S,NaN,"England / Bennington, VT"
1,2,1,"Wells, Miss. Joan",female,4.0,1,1,29103,23.0000,NaN,S,NaN,"Cornwall / Akron, OH"
2,2,1,"Duran y More, Miss. Florentina",female,30.0,1,0,SC/PARIS 2148,13.8583,NaN,C,NaN,"Barcelona, Spain / Havana, Cuba"
3,3,0,"Scanlan, Mr. James",male,NaN,0,0,36209,7.7250,NaN,Q,NaN,NaN
4,3,1,"Bradley, Miss. Bridget Delia",female,22.0,0,0,334914,7.7250,NaN,Q,NaN,"Kingwilliamstown, Co Cork, Ireland Glens Falls..."


In [966]:
type(df_test)
df_test.columns

Index(['pclass', 'survived', 'name', 'sex', 'age', 'sibsp', 'parch', 'ticket',
       'fare', 'cabin', 'embarked', 'body', 'home.dest'],
      dtype='object')

In [967]:
df_train.describe()

,pclass,survived,age,sibsp,parch,fare,body
count,916.000000,916.000000,741.000000,916.000000,916.000000,916.000000,85.000000
mean,2.292576,0.385371,30.231444,0.532751,0.371179,32.532050,157.870588
std,0.842576,0.486949,13.934240,1.125957,0.837508,47.416524,98.257653
min,1.000000,0.000000,0.170000,0.000000,0.000000,0.000000,1.000000
25%,1.000000,0.000000,21.000000,0.000000,0.000000,7.895800,68.000000
50%,3.000000,0.000000,28.000000,0.000000,0.000000,14.454200,148.000000
75%,3.000000,1.000000,39.000000,1.000000,0.000000,31.275000,249.000000
max,3.000000,1.000000,74.000000,8.000000,9.000000,512.329200,328.000000


##### 데이터 피처 설명
- pclass : Passenger Class, 승객 등급
- survived : 생존 여부 : target 값이 됨.
- name : 승객 이름
- sex : 승객 성별
- age : 승객 나이
- sibsp : 탑승 한 형제/배우자 수
- parch : 탑승 한 부모/자녀 수
- ticket : 티켓 번호
- fare : 승객 지불 요금
- cabin : 선실 이름
- embarked : 승선항 (C = 쉘 부르그, Q = 퀸즈타운, S = 사우스 햄튼)
- body : 사망자 확인 번호 - 분석과정에서 제외해야 함. 머신러닝에 100% 영향을 미침.
- home.dest : 고향/목적지

In [968]:
df_train.info() # nan : age, cabin, body, home.dest, name, embarked
# target : survived
#  bode, name 삭제

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 916 entries, 0 to 915
Data columns (total 13 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   pclass     916 non-null    int64  
 1   survived   916 non-null    int64  
 2   name       916 non-null    object 
 3   sex        916 non-null    object 
 4   age        741 non-null    float64
 5   sibsp      916 non-null    int64  
 6   parch      916 non-null    int64  
 7   ticket     916 non-null    object 
 8   fare       916 non-null    float64
 9   cabin      214 non-null    object 
 10  embarked   914 non-null    object 
 11  body       85 non-null     float64
 12  home.dest  527 non-null    object 
dtypes: float64(3), int64(4), object(6)
memory usage: 93.2+ KB


# nan :  age, cabin, body, home.dest, name, embarked

In [969]:
df_train['embarked']= (df_train['embarked'].fillna(0))

In [970]:
df_train['embarked'].isnull().value_counts

<bound method IndexOpsMixin.value_counts of 0      False
1      False
2      False
3      False
4      False
       ...  
911    False
912    False
913    False
914    False
915    False
Name: embarked, Length: 916, dtype: bool>

In [971]:
df_train[df_train['embarked'].isnull() == True] 

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,body,home.dest


In [972]:
df_train['age']= (df_train['age'].fillna(0))

In [973]:
# nan : age
df_train[df_train['age'].isnull() == True]

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,body,home.dest


In [974]:
df_train.columns

Index(['pclass', 'survived', 'name', 'sex', 'age', 'sibsp', 'parch', 'ticket',
       'fare', 'cabin', 'embarked', 'body', 'home.dest'],
      dtype='object')

In [975]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 916 entries, 0 to 915
Data columns (total 13 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   pclass     916 non-null    int64  
 1   survived   916 non-null    int64  
 2   name       916 non-null    object 
 3   sex        916 non-null    object 
 4   age        916 non-null    float64
 5   sibsp      916 non-null    int64  
 6   parch      916 non-null    int64  
 7   ticket     916 non-null    object 
 8   fare       916 non-null    float64
 9   cabin      214 non-null    object 
 10  embarked   916 non-null    object 
 11  body       85 non-null     float64
 12  home.dest  527 non-null    object 
dtypes: float64(3), int64(4), object(6)
memory usage: 93.2+ KB


In [976]:
df_train

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,body,home.dest
0,2,1,"Mellinger, Miss. Madeleine Violet",female,13.00,0,1,250644,19.5000,NaN,S,NaN,"England / Bennington, VT"
1,2,1,"Wells, Miss. Joan",female,4.00,1,1,29103,23.0000,NaN,S,NaN,"Cornwall / Akron, OH"
2,2,1,"Duran y More, Miss. Florentina",female,30.00,1,0,SC/PARIS 2148,13.8583,NaN,C,NaN,"Barcelona, Spain / Havana, Cuba"
3,3,0,"Scanlan, Mr. James",male,0.00,0,0,36209,7.7250,NaN,Q,NaN,NaN
4,3,1,"Bradley, Miss. Bridget Delia",female,22.00,0,0,334914,7.7250,NaN,Q,NaN,"Kingwilliamstown, Co Cork, Ireland Glens Falls..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
911,3,1,"Dean, Miss. Elizabeth Gladys ""Millvina""",female,0.17,1,2,C.A. 2315,20.5750,NaN,S,NaN,"Devon, England Wichita, KS"
912,3,0,"Guest, Mr. Robert",male,0.00,0,0,376563,8.0500,NaN,S,NaN,NaN
913,3,1,"Smyth, Miss. Julia",female,0.00,0,0,335432,7.7333,NaN,Q,NaN,NaN
914,2,1,"Sincock, Miss. Maude",female,20.00,0,0,C.A. 33112,36.7500,NaN,S,NaN,"Cornwall / Hancock, MI"


In [977]:
df_train['age']

0      13.00
1       4.00
2      30.00
3       0.00
4      22.00
       ...  
911     0.17
912     0.00
913     0.00
914    20.00
915    32.00
Name: age, Length: 916, dtype: float64

In [978]:
df_train.columns

Index(['pclass', 'survived', 'name', 'sex', 'age', 'sibsp', 'parch', 'ticket',
       'fare', 'cabin', 'embarked', 'body', 'home.dest'],
      dtype='object')

In [979]:
pclass= df_train['pclass']
pclass

0      2
1      2
2      2
3      3
4      3
      ..
911    3
912    3
913    3
914    2
915    3
Name: pclass, Length: 916, dtype: int64

In [980]:
# name, ticket, body, cabin, hone.dest 삭제
a_p = df_train.iloc[:,4:7]
a_p

,age,sibsp,parch
0,13.00,0,1
1,4.00,1,1
2,30.00,1,0
3,0.00,0,0
4,22.00,0,0
...,...,...,...
911,0.17,1,2
912,0.00,0,0
913,0.00,0,0
914,20.00,0,0


In [981]:
df_train.columns

Index(['pclass', 'survived', 'name', 'sex', 'age', 'sibsp', 'parch', 'ticket',
       'fare', 'cabin', 'embarked', 'body', 'home.dest'],
      dtype='object')

In [982]:
fare  = df_train['fare']

In [983]:
df_train['embarked'].unique()

array(['S', 'C', 'Q', 0], dtype=object)

In [984]:
df_train.head(5)

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,body,home.dest
0,2,1,"Mellinger, Miss. Madeleine Violet",female,13.0,0,1,250644,19.5000,NaN,S,NaN,"England / Bennington, VT"
1,2,1,"Wells, Miss. Joan",female,4.0,1,1,29103,23.0000,NaN,S,NaN,"Cornwall / Akron, OH"
2,2,1,"Duran y More, Miss. Florentina",female,30.0,1,0,SC/PARIS 2148,13.8583,NaN,C,NaN,"Barcelona, Spain / Havana, Cuba"
3,3,0,"Scanlan, Mr. James",male,0.0,0,0,36209,7.7250,NaN,Q,NaN,NaN
4,3,1,"Bradley, Miss. Bridget Delia",female,22.0,0,0,334914,7.7250,NaN,Q,NaN,"Kingwilliamstown, Co Cork, Ireland Glens Falls..."


In [985]:
df_train['embarked'].head(5)

0    S
1    S
2    C
3    Q
4    Q
Name: embarked, dtype: object

In [986]:
type(df_train['embarked'])

pandas.core.series.Series

In [987]:
df_train['embarked']=df_train['embarked'].fillna(0)
df_train['sex']= df_train['sex'].fillna(0)

In [988]:
df_train['sex'].isnull()

0      False
1      False
2      False
3      False
4      False
       ...  
911    False
912    False
913    False
914    False
915    False
Name: sex, Length: 916, dtype: bool

In [989]:
embarked = pd.get_dummies(df_train['embarked'])

In [990]:
df_train['embarked'].isnull()

0      False
1      False
2      False
3      False
4      False
       ...  
911    False
912    False
913    False
914    False
915    False
Name: embarked, Length: 916, dtype: bool

In [991]:
df_train['embarked'].unique()

array(['S', 'C', 'Q', 0], dtype=object)

In [992]:
df_train['sex'].unique()

array(['female', 'male'], dtype=object)

In [993]:
sex = pd.DataFrame({'sex':['female', 'male']})
sex

,sex
0,female
1,male


In [994]:
embarked = pd.DataFrame({'embarked':['S', 'C', 'Q', 0]})
embarked

,embarked
0,S
1,C
2,Q
3,0


In [995]:
target = df_train.iloc[:,1:2]
data = pd.concat([pclass,sex,a_p,fare,embarked],axis=1)

In [996]:
df_train.iloc[:,1:2]

,survived
0,1
1,1
2,1
3,0
4,1
...,...
911,1
912,0
913,1
914,1


In [997]:
df_train.columns 
# 'pclass','sex', 'age', 'sibsp', 'parch','fare','embarked',

Index(['pclass', 'survived', 'name', 'sex', 'age', 'sibsp', 'parch', 'ticket',
       'fare', 'cabin', 'embarked', 'body', 'home.dest'],
      dtype='object')

In [998]:
data

,pclass,sex,age,sibsp,parch,fare,embarked
0,2,female,13.00,0,1,19.5000,S
1,2,male,4.00,1,1,23.0000,C
2,2,NaN,30.00,1,0,13.8583,Q
3,3,NaN,0.00,0,0,7.7250,0
4,3,NaN,22.00,0,0,7.7250,NaN
...,...,...,...,...,...,...,...
911,3,NaN,0.17,1,2,20.5750,NaN
912,3,NaN,0.00,0,0,8.0500,NaN
913,3,NaN,0.00,0,0,7.7333,NaN
914,2,NaN,20.00,0,0,36.7500,NaN


In [999]:
data = np.array(data)
target = np.array(target)


In [1000]:
target

array([[1],
       [1],
       [1],
       [0],
       [1],
       [0],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [1],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [0],
       [1],
       [0],
    

In [1001]:
# knn, 로지스틱, 랜덤포레스트

In [1002]:
# 로지스틱
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(
  target, data,random_state=42)




In [1003]:
train_input = train_input.reshape(-1,1)

In [1033]:
test_input = test_input.reshape(-1,1)
train_target = np.array(train_target)


In [1005]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
ss.fit(train_input)
train_scaled = ss.transform(train_input)
test_scaled = ss.transform(test_input)

In [1037]:
lr = LogisticRegression()
lr.fit(train_scaled,train_target)

c:\Python312\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


ValueError: Input contains NaN